In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# ИСПРАВЛЕННЫЕ НАСТРОЙКИ
batch_size = 32
img_height = 224  # Стандарт для MobileNetV2 вместо 512!
img_width = 224
epochs = 30  # Уменьшаем количество эпох
directory = 'Hot_Dogs_Photo'

print(f"Используем разрешение: {img_height}x{img_width}")
print(f"Batch size: {batch_size}")
print(f"Epochs: {epochs}")

In [ ]:
# Создаем датасеты с правильным разрешением
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset="training",
    crop_to_aspect_ratio=True,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset="validation",
    crop_to_aspect_ratio=True,
)

# Получаем названия классов
class_names = train_ds.class_names
print("Классы:", class_names)
print(f"Количество классов: {len(class_names)}")

In [ ]:
# ИСПРАВЛЕННАЯ МОДЕЛЬ С АУГМЕНТАЦИЕЙ ДАННЫХ
model = Sequential([
    # АУГМЕНТАЦИЯ ДАННЫХ (ОЧЕНЬ ВАЖНО!)
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
    
    # Предобработка
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    
    # Transfer learning с MobileNetV2
    keras.applications.MobileNetV2(
        input_shape=(img_height, img_width, 3),
        include_top=False,
        weights='imagenet'
    ),
    
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(len(class_names))
])

# Замораживаем базовую модель
model.layers[5].trainable = False  # MobileNetV2 слой

model.summary()

In [ ]:
# Компиляция с правильными настройками
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Оптимизация производительности
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# ОБУЧЕНИЕ С CALLBACKS
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )
]

print(f"Начинаем обучение на {epochs} эпохах...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

In [ ]:
# ГРАФИКИ РЕЗУЛЬТАТОВ
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.tight_layout()
plt.savefig('training_results_224_corrected.png', dpi=300, bbox_inches='tight')
plt.show()

# Финальные метрики
final_train_acc = acc[-1]
final_val_acc = val_acc[-1]
print(f"\nФинальная точность обучения: {final_train_acc:.4f}")
print(f"Финальная точность валидации: {final_val_acc:.4f}")
print(f"Разница (переобучение): {abs(final_train_acc - final_val_acc):.4f}")

In [ ]:
# СОХРАНЕНИЕ МОДЕЛИ
model.save('src/main/resources/model/corrected_model.keras')
print("Модель сохранена как corrected_model.keras")

# Сохраняем классы
with open('src/main/resources/model/classes_corrected.txt', 'w') as f:
    for class_name in class_names:
        f.write(f"{class_name}\n")
print("Классы сохранены")

In [ ]:
# ТЕСТИРОВАНИЕ МОДЕЛИ
def predict_image(image_path):
    """Предсказание для одного изображения"""
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)
    
    return predicted_class, confidence

# Пример использования
try:
    predicted_class, confidence = predict_image('4.jpg')
    print(f"Предсказание: {predicted_class} с уверенностью {confidence:.2f}%")
except FileNotFoundError:
    print("Тестовое изображение 4.jpg не найдено")
    print("Попробуйте другое изображение из датасета")